In [45]:
import sys
sys.path.append('..')

from sqlalchemy import create_engine
import geopandas as gpd
import numpy as np
import pandas as pd

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

In [22]:
datasets = {
    'canha_quemada' : 377418,
    'credito_siembra' : 360179
}
datasets

In [4]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )
    
def get_catastro():
    engine = obtener_engine()
    try:
        query = """
            SELECT idd, geom, unidad_01, unidad_02, 
                unidad_03, unidad_04, unidad_05, 
                variedad, soca, textura, cultivo, area 
            FROM catastro_iag.catastro order by idd
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error get catastro: {e}")
        return gpd.GeoDataFrame()
    return None

def get_catastro_sync():
    engine = obtener_engine()
    try:
        query = """
            SELECT idd, geom, unidad_01, unidad_02, 
                unidad_03, unidad_04, unidad_05, 
                variedad, soca, textura, cultivo, area 
            FROM amigcloud_sync.catastro_amigocloud_sync order by idd
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error get catastro: {e}")
        return gpd.GeoDataFrame()
    return None

In [10]:
gdf_catastro = get_catastro()
gdf_catastro_sync = get_catastro_sync()

In [20]:
# identificar registros nuevos
reg_nuevos = gdf_catastro[~gdf_catastro['idd'].isin(gdf_catastro_sync['idd'])]

# completa campos para registro
data_reg_nuevos = reg_nuevos[['idd']].copy()
data_reg_nuevos['operacion'] = 'C'
data_reg_nuevos['procesado'] = 'false'

# iterar en los datasets
all_reg_nuevos = None
for key in datasets:
    data_reg_nuevos['dataset'] = datasets[key]
    all_reg_nuevos = pd.concat([all_reg_nuevos, data_reg_nuevos], ignore_index=True)

# registro en base de datos
all_reg_nuevos.to_sql("catastro_sync", obtener_engine(), schema="amigcloud_sync", if_exists="append", index=False)

In [53]:
# identificar registros eliminados
reg_eliminados = gdf_catastro_sync[~gdf_catastro_sync['idd'].isin(gdf_catastro['idd'])]

# completa campos para registro
data_reg_eliminados = reg_eliminados[['idd']].copy()
data_reg_eliminados['operacion'] = 'D'
data_reg_eliminados['procesado'] = 'false'

# iterar en los datasets
all_reg_eliminados = None
for key in datasets:
    data_reg_eliminados['dataset'] = datasets[key]
    all_reg_eliminados = pd.concat([all_reg_eliminados, data_reg_eliminados], ignore_index=True)

# registro en base de datos
all_reg_eliminados.to_sql("catastro_sync", obtener_engine(), schema="amigcloud_sync", if_exists="append", index=False)

In [63]:
reg_comparacion = gdf_catastro_sync.merge(gdf_catastro, on='idd', how='inner', suffixes=('_cat', '_cat_sync'))
reg_modificados = reg_comparacion[
    (reg_comparacion['unidad_01_cat'] != reg_comparacion['unidad_01_cat_sync']) |
    (reg_comparacion['unidad_02_cat'] != reg_comparacion['unidad_02_cat_sync']) |
    (reg_comparacion['unidad_03_cat'] != reg_comparacion['unidad_03_cat_sync']) |
    (reg_comparacion['unidad_04_cat'] != reg_comparacion['unidad_04_cat_sync']) |
    (reg_comparacion['unidad_05_cat'] != reg_comparacion['unidad_05_cat_sync']) |
    (reg_comparacion['variedad_cat']  != reg_comparacion['variedad_cat_sync'])  |
    (reg_comparacion['soca_cat']      != reg_comparacion['soca_cat_sync'])      |
    (reg_comparacion['textura_cat']   != reg_comparacion['textura_cat_sync'])   |
    (reg_comparacion['cultivo_cat']   != reg_comparacion['cultivo_cat_sync'])   |
    (np.round(reg_comparacion['area_cat'], 1) != np.round(reg_comparacion['area_cat_sync'], 1))
]

len(reg_modificados)

2971

In [65]:
# completa campos para registro
data_reg_modificados = reg_modificados[['idd']].copy()
data_reg_modificados['operacion'] = 'U'
data_reg_modificados['procesado'] = 'false'

# iterar en los datasets
all_reg_modificados = None
for key in datasets:
    data_reg_modificados['dataset'] = datasets[key]
    all_reg_modificados = pd.concat([all_reg_modificados, data_reg_modificados], ignore_index=True)

# registro en base de datos
all_reg_modificados.to_sql("catastro_sync", obtener_engine(), schema="amigcloud_sync", if_exists="append", index=False)